In [1]:
%matplotlib inline
import librosa, librosa.display
import IPython.display as ipd
import vamp
import matplotlib.pyplot as plt
import numpy as np
import scipy.fftpack as fft
import re

In [2]:
import re
import collections

In [3]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pprint = pp.pprint

In [4]:
import vamp

In [5]:
#path = '/home/sonja/Dropbox/RC/passive_ear_trainer/Songs/internationale.mp4'
path = '/home/sonja/Dropbox/RC/passive_ear_trainer/Songs/Autumn Leaves - Yenne Lee plays 2004 Pepe Romero Jr.-HxGT5z6d-GA.m4a'
#path = '/home/sonja/Dropbox/RC/passive_ear_trainer/Songs/Beatles - I wanna hold your hand - live.m4a'
#path = '/home/sonja/Dropbox/RC/passive_ear_trainer/Songs/Beatles - I wanna hold your hand -studio.m4a'
#path = '/home/sonja/Dropbox/RC/passive_ear_trainer/Songs/Dumb Ways to Die-IJNR2EpS0jw.m4a'
#path = '/home/sonja/Dropbox/RC/passive_ear_trainer/Songs/Queen - I Want to Break Free (Official Lyric Video)-WUOtCLOXgm8.m4a'
#path = '/home/sonja/Dropbox/RC/passive_ear_trainer/Songs/Selena Gomez, Marshmello - Wolves (Official Music Video)-cH4E_t3m3xM.m4a'
#path = '/home/sonja/Dropbox/RC/passive_ear_trainer/Songs/Ska-p Los hijos bastardos de la globalizacion con Letra-upnPasIYeMc.m4a'
#path = '/home/sonja/Dropbox/RC/passive_ear_trainer/Songs/The Killers - Human-RIZdjT1472Y.m4a'
#path = '/home/sonja/Dropbox/RC/passive_ear_trainer/Songs/THE MUFFIN SONG (asdfmovie feat. Schmoyoho)-LACbVhgtx9I.m4a'

In [6]:
#HAS TO BE RUN FOR EVERY SONG
song = librosa.load(path)

In [7]:
cow = librosa.load('/home/sonja/Dropbox/RC/passive_ear_trainer/Songs/Kuh muht-TdheW61w4Co.m4a')

In [8]:
song_analysis = vamp.collect(song[0], song[1], "nnls-chroma:chordino")['list']
pprint(list(enumerate(song_analysis)))

[   (0, {'label': 'N', 'timestamp':  0.371519274}),
    (1, {'label': 'F#m7b5', 'timestamp':  0.464399092}),
    (2, {'label': 'Bm', 'timestamp':  4.922630385}),
    (3, {'label': 'Cmaj7', 'timestamp':  7.894784580}),
    (4, {'label': 'D', 'timestamp':  9.287981859}),
    (5, {'label': 'Am', 'timestamp':  11.238458049}),
    (6, {'label': 'B7', 'timestamp':  12.074376417}),
    (7, {'label': 'Em', 'timestamp':  14.489251700}),
    (8, {'label': 'Am7', 'timestamp':  19.319002267}),
    (9, {'label': 'D', 'timestamp':  21.640997732}),
    (10, {'label': 'Cmaj7', 'timestamp':  25.634829931}),
    (11, {'label': 'Am7', 'timestamp':  27.585306122}),
    (12, {'label': 'B', 'timestamp':  29.535782312}),
    (13, {'label': 'Em', 'timestamp':  31.114739229}),
    (14, {'label': 'F#m7', 'timestamp':  35.665850340}),
    (15, {'label': 'B7', 'timestamp':  37.894965986}),
    (16, {'label': 'Em', 'timestamp':  39.845442176}),
    (17, {'label': 'Am7', 'timestamp':  43.932154195}),
    (18, {'lab

In [9]:
ENCR_NOTES = {'C':0, 'C#':1, 'Db':1, 'D':2, 'D#':3, 'Eb':3, 'E':4, 'Fb':4, 'E#':5, 'F':5, 'F#':6, 'Gb':6, 'G':7, 'G#':8, 'Ab':8, 'A':9, 'A#':10, 'Bb':10, 'B':11, 'Cb':11, 'B#':0}

DECR_FLAT_NOTES = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
DECR_SHARP_NOTES = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
DECR_NOTES_PAIRS = [('C'), ('Db', 'C#'), ('D'), ('Eb', 'D#'), ('E'), ('F'), ('Gb', 'F#'), ('G'), ('Ab', 'G#'), 'A', ('Bb', 'A#'), ('B')]

## Parse chords:

In [10]:
def extract_root(chord):
    root = chord[:2] if (len(chord) > 1 and chord[1] in ['b', '#']) else chord[0]
    if root =="N":
        return
    beginning_next_token = len(root)
    if root[0] in ['A', 'B', 'C', 'D', 'E', 'F', 'G']:
        return root, chord[beginning_next_token:]
    else:
        raise Exception("This chord doesn't have a root note.")

In [11]:
def extract_third_mod(piece_chord):
    third = "major"
    length_token = 0
    if piece_chord:
        if piece_chord[0] == 'm' and piece_chord[:3] != "maj":
            third = 'minor'
            length_token = 1
        elif piece_chord[:3] == "sus":
            third = piece_chord[: 4]
            length_token = 4
    return third, piece_chord[length_token:]

In [12]:
def extract_fifth_mod(piece_chord):
    fifth = "perfect"
    length_token = 0
    if piece_chord and piece_chord[:3] in ["dim", "aug"]:
        fifth = piece_chord[:3]
        length_token = 3
    return fifth, piece_chord[length_token:]

In [13]:
extract_fifth_mod('maj7b10/G#')

('perfect', 'maj7b10/G#')

In [14]:
def extract_bass(root, piece_chord):
    bass_regex = re.compile(r"\/([A-G](?:#|b)?)$")
    bass_specification = re.findall(bass_regex, piece_chord)
    if bass_specification:
        bass = bass_specification[0]
        rest =  piece_chord[: -(len(bass) + 1)]
    else:
        bass = root
        rest = piece_chord
    return bass, rest

In [15]:
extract_bass("D","sus2maj7b10/G#")

('G#', 'sus2maj7b10')

In [16]:
def extract_seventh_mod(piece_chord):
    seventh = None 
    re_others = re.compile(r"(?:maj)?(?:7)")
    seventh = re.findall(re_others, piece_chord)
    meaning_abbreviations = {"maj7" : "major", "7" : "minor"}
    if seventh:
        return meaning_abbreviations[seventh[0]]
    else:
        return
    

In [17]:
extract_seventh_mod("bamaj7b10/G#")

'major'

In [18]:
def get_notes(chord):
    seminotes_third = {"major" : 4, "minor" : 3, "sus2" : 2, "sus4" : 5}
    seminotes_fifth = {"perfect" : 7, "dim" : 6, "aug" : 8}
    seminotes_seventh = {"major" : 11, "minor" : 10}
    root, rest = extract_root(chord)
    third_mod, rest = extract_third_mod(rest)
    fifth_mod, rest = extract_fifth_mod(rest)
    bass, rest = extract_bass(root, rest)
    seventh_mod = extract_seventh_mod(rest)
    notes = {ENCR_NOTES[root], (ENCR_NOTES[root] + seminotes_third[third_mod]) % 12, (ENCR_NOTES[root] + seminotes_fifth[fifth_mod]) % 12}
    if bass:
        notes.add(ENCR_NOTES[bass])
    if seventh_mod:
        notes.add(((ENCR_NOTES[root] + seminotes_seventh[seventh_mod])) % 12)
    return notes

In [19]:
get_notes("Bbmmaj7")

{1, 5, 9, 10}

In [20]:
def get_major_scale(root):
    return (map(lambda rel_note: (root + rel_note) % 12, [0, 2 ,4 ,5 ,7 ,9, 11]))

In [21]:
MAJOR_SCALES = {tuple(get_major_scale(ENCR_NOTES[root])) : root for root in ENCR_NOTES.keys()}

In [22]:
# #parameter: chord after removing root and maybe third and fifth specification and bass note
# def get_rel_seventh_and_others(piece_chord):
#     seminotes_from_root = {"major" : 12, "minor" : 11}
#     seventh = "minor"
#     length_token = 0
#     re_others = re.compile(r"(maj|b)([1-9]?[0-9])")
#     extra_notes = re.findall(re_others, piece_chord)
    
#     pointer += sum(map((lambda extra : len(extra)), parsed["extra_notes"]))
    
    
    
#     if piece_chord and piece_chord[:3] == "maj":
#         if piece_chord[:3] == "maj7":
#             seventh = "major"
#             length_token = 4
        

In [25]:
extract_root("Bbmmaj7")

('Bb', 'mmaj7')

In [26]:
extract_third_mod('mmaj7')

('minor', 'maj7')

In [27]:
extract_fifth_mod('augmaj7')

('aug', 'maj7')

In [28]:
def chord_arithmetics(parsed_chord):
    root = ENCR_NOTES.get(parsed_chord["root"])
    if not root:
        print("something is ")

In [29]:
def extend_short_rec(short_rec, desired_num_samples, desired_vol):
    bounds = [1113687, 1148687] #FIXME
    current_vol = max(short_rec[0])
    
    sample = last_not_0_before_par(short_rec, bounds[1])
    for _ in range(50):
        sample = sign_change(short_rec, sample-1)
    end_copy = sample

    for _ in range(1000):
        sample = sign_change(short_rec, sample-1)
    beginning_copy = sample

    extension = short_rec[0][bounds[0]:end_copy]
    while len(extension) < desired_num_samples:
        extension = np.concatenate((extension, short_rec[0][beginning_copy:end_copy]))
        
    extension = extension/current_vol*desired_vol
    
    return extension[:desired_num_samples]

In [30]:
def cow_generator(amp, rate):
    if rate != cow[1]:
        return False
    def f(num_samples):
        return extend_short_rec(cow, num_samples, amp)
    return f 

In [31]:
def last_not_0_before_par(rec, par):
    ind = par
    while rec[0][ind] == 0:
        ind -= 1
    return ind

def first_0_before_par(rec, par):
    ind = par
    while rec[0][ind] != 0:
        ind -= 1
    return ind

def sign_change(rec, par):
    ind = par
    while rec[0][ind] * rec[0][par] > 0:
        ind -= 1
    return ind

In [32]:
def note_by_num_samples(freq, num_samples, amp=1, rate=song[1]):
    data = np.array([np.sin(2*np.pi*freq*librosa.samples_to_time(sample, rate))*amp for sample in range(num_samples)], dtype=np.float32)
    return data

In [33]:
def synth_generator(freq, amp, rate):
    def f(num_samples):
        return note_by_num_samples(freq, num_samples, amp, rate)
    return f

In [34]:
scale_deg_in_semitones = {1:0, 2:2, 3:4, 4:5, 5:7, 6:9, 7:11}

In [35]:
#given a tonality and a scale degree, find the chord corresponding to the scale degree in the given tonality
def absolute_chord(tonality, scale_deg):
    encr_chord = (ENCR_NOTES[tonality] + scale_deg_in_semitones[scale_deg]) % 12
    if len(tonality) > 1 and tonality[1] == 'b':
        chord = DECR_FLAT_NOTES[encr_chord]
    else:
        chord = DECR_SHARP_NOTES[encr_chord]
    return chord

In [36]:
bass_freq = {'C':65.41, 'C#':69.30, 'D':73.42, 'D#':77.78, 'E':82.41, 'F':87.31, 'F#':92.50, 'G':98.00, 'G#':103.83, 'A':110.00, 'A#':116.54, 'B':123.47}
high_pitch_freq = {'C':130.81, 'C#':138.59, 'D':146.83, 'D#':155.56, 'E':164.81, 'F':174.61, 'F#':185.00, 'G':196.00, 'G#':207.65, 'A':220.00, 'A#':233.08, 'B':246.94}

In [37]:
def find_bounds_samples(song_analysis, chord):
    #bounds_chord = [librosa.time_to_samples(t, song[1]) for t in timestamps_chord]

    pairs = list(zip(song_analysis[:-1], song_analysis[1:]))
    time_bounds = [(float(item1['timestamp']), float(item2['timestamp'])) for item1, item2 in pairs if re.match(re.escape(chord)+r'.*', item1['label'])]
    bounds = [librosa.time_to_samples(t, song[1])for t in time_bounds]
    return bounds

In [38]:
def find_amplitude_rel_song(song, strength):
    song_intensity = max(song[0])
    return song_intensity*strength

In [39]:
def generate_audio_to_add(song_analysis, song_length, chord, timber, amp, rate):
    if timber == 'cow':
        note_generator = cow_generator(amp, rate)
        if not note_generator:
            'Sorry, the cow is tired of mooing. Please, choose a different timber!'
    else:
        if timber == 'low_pitch_synth':
            freq = bass_freq[chord]
        else:
            freq = high_pitch_freq[chord]
        note_generator = synth_generator(freq, amp, rate)
    
    bounds_chord = find_bounds_samples(song_analysis, chord)
    add_to_song = np.zeros(song_length)
    for beginning, end in bounds_chord:
        if end > song_length:
            end = song_length
        add = note_generator(end - beginning)
        add_to_song[beginning:end] = add
        
    return add_to_song

In [40]:
def get_tonality_from_dominant(dominant):
    if len(dominant) > 1 and dominant[1] == 'b':
        decr = DECR_FLAT_NOTES
    else:
        decr = DECR_SHARP_NOTES
    arithm_repr_root = ( ENCR_NOTES[dominant] - 7 ) % 12
    return decr[arithm_repr_root]

In [47]:
def find_tonality(song_analysis):
    #look for a dominant chord: TO DO
    
    #find last chord
    last_chord_index = -1
    while song_analysis[last_chord_index]['label'] == 'N':
        last_chord_index -= 1
    last_chord = song_analysis[last_chord_index]['label']
    last_root = extract_root(last_chord)[0]
    
    return last_root
    
    # find the chord that is played the most
    roots_info = [extract_root(chord['label']) for chord in song_analysis if extract_root(chord['label'])]
    roots = [root for root, _ in roots_info]
    numb_appearance_root = collections.Counter(roots)
    most_appearing_root = numb_appearance_root.most_common(1)[0][0]
    
    
    #find tonality via dominant7 chords:
    maj_dominant7_chords = {root for root, rest in roots_info if extract_third_mod(rest)[0] == "major" and extract_seventh_mod(rest) == "minor"}
    potential_min_dominant7_chords = {root for root, rest in roots_info if extract_third_mod(rest)[0] == "minor" and extract_seventh_mod(rest) == "minor"}

    tonalities = {}
    if maj_dominant7_chords:
        tonalities = {get_tonality_from_dominant(dominant) for dominant in maj_dominant7_chords if get_tonality_from_dominant(dominant) in roots}
    return tonalities
    #find possible scales:
    index = 0
    while song_analysis[index]['label'] == 'N':
        index += 1
    first_chord = song_analysis[index]['label']
    old_possible_scales = new_possible_scales = {scale : root for scale, root in MAJOR_SCALES.items() if get_notes(first_chord).issubset({note for note in scale})}
    
    while index < len(song_analysis) and new_possible_scales:
        index += 1
        chord = song_analysis[index]['label']
        if chord == 'N':
            continue
        old_possible_scales = new_possible_scales
        new_possible_scales = {scale : root for scale, root in old_possible_scales.items() if get_notes(chord).issubset({note for note in scale})}
    #return old_possible_scales, index
    

        

#     sign = ''
#     if len(most_appearing_chord) > 1:
#         if most_appearing_chord[1] == '#' or most_appearing_chord[1] == 'b':
#             sign = most_appearing_chord[1]

#     tonality = most_appearing_chord[0] + sign
    tonality = most_appearing_root
    
#     certainty = True
#     if last_chord != most_appearing_chord:
#         certeinty = False
        
    return 

In [42]:
roots_info = [extract_root(chord['label']) for chord in song_analysis if extract_root(chord['label'])]
rest = roots_info[5][1]
print(rest)



7


In [43]:
find_tonality(song_analysis)

{'B', 'E', 'F#', 'G'}

In [44]:
maj_dominant7_chords = {'B', 'C#', 'D', 'F#'}
print({get_tonality_from_dominant(dominant) for dominant in maj_dominant7_chords})

{'E', 'B', 'F#', 'G'}


In [48]:
scale_deg = 1 # 1,2,3,4,5,6 or 7
timber = 'high_pitch_synth' # 'low_pitch_synth', 'high_pitch_synth' or 'cow'
amp = 1 #1/4, 1/2, 1, 2 

tonality = find_tonality(song_analysis)
# if not certainty:
#     print('The tonality is probably wrong, sorry for that!')

chord = absolute_chord(tonality, scale_deg)
length = len(song[0])
to_add = generate_audio_to_add(song_analysis, length, chord, timber, amp, song[1])



In [46]:
output = song[0] + to_add
print(len(to_add))
print(len(song[0]))
print(tonality)
bounds = find_bounds_samples(song_analysis, tonality)
beginning, end = bounds[4]
ipd.Audio(output, rate=song[1])

NameError: name 'to_add' is not defined

In [ ]:
output = song[0] + to_add
print(len(to_add))
print(len(song[0]))
print(tonality)
bounds = find_bounds_samples(song_analysis, tonality)
beginning, end = bounds[4]
ipd.Audio(output, rate=song[1])

The rest is not necessary anymore

In [ ]:
#HAS TO BE RUN FOR EVERY SONG
#Compute the chord that corresponds to a certain scale degree:
scale_degree = 1
chord = notes_sharp_medium_pitch[(ENCR_NOTES[tonality] + scale_degree_in_semitones[scale_degree]) % 12]
print(chord)

In [ ]:
#HAS TO BE RUN FOR EVERY SONG
#Find the time stamps in between which the chord corresponding to the chosen scale degree appears in the song:
chord_pairs = list(zip(chords[:-1], chords[1:]))
timestamps_chord = [ (float(c[0]['timestamp']), float(c[1]['timestamp'])) for c in chord_pairs if re.match(re.escape(chord[0])+r'.*', c[0]['label'])]
bounds_chord = [librosa.time_to_samples(t, song[1]) for t in timestamps_chord]
lables_chord = [c[0]['label'] for c in chord_pairs if re.match(re.escape(chord[0])+r'.*', c[0]['label'])]

#chord_number = 2
#print(lables_chord[chord_number])
#ipd.Audio(song[0][bounds_chord[chord_number][0]:bounds_chord[chord_number][1]], rate=song[1])


In [ ]:
#Has TO BE TUN FOR EVERY SONG
#Compute highest amplitude in the song:
max_amplitude = max(song[0])
print(max_amplitude)

In [ ]:
#HAS TO BE RUN FOR EVERY SONG
#Generate the output (audio of song plus the base note of the chord corresponding to the scale degree chosen every time that chord appears):
add_to_song = np.zeros(len(song[0]))
for beginning, end in bounds_chord:
    if end > len(song[0]):
        end = len(song[0])
    add = note_by_num_samples(chord[1] , end-beginning, amp=(max_amplitude)*5)
    add_to_song[beginning:end] = add
    
output = song[0] + add_to_song
ipd.Audio(output, rate=song[1])

In [ ]:
add_cow = np.zeros(len(song[0]))
max_num_samples = max([end-beginning for beginning, end in bounds_chord])
long_cow = long_enough_ignoring_frequencies(max_num_samples)
for beginning, end in bounds_chord:
    if end > len(song[0]):
        end = len(song[0])
    add = long_cow[:(end-beginning)]
    add_cow[beginning:end] = add

output = song[0] + add_cow
ipd.Audio(output, rate=song[1])

In [ ]:
#Generate the output (audio of song plus the base note of the chord corresponding to the scale degree chosen every time that chord appears):
add_to_song = np.zeros(len(song[0]))
for beginning, end in bounds_chord:
    if end > len(song[0]):
        end = len(song[0])
    add = note_by_num_samples(chord[1] , end-beginning, amp=max_amplitude)
    add_to_song[beginning:end] = add
    
output = song[0] + add_to_song
ipd.Audio(output, rate=song[1])

In [ ]:
#computations for for getting the bounds of the cow sample between which the cow moos
cow_chords = vamp.collect(cow[0], cow[1], "nnls-chroma:chordino")['list']
label = 'N'
ind = 0
while label == 'N':
    ind += 1
    label = cow_chords[ind]['label']
beginning_ind = ind
print(beginning_ind)
while label != 'N':
    ind += 1
    label = cow_chords[ind]['label']
end_ind = ind - 1
print(end_ind)

timestamps_cow = [float(cow_chords[beginning_ind]['timestamp']), float(cow_chords[2]['timestamp'])]
bounds_cow = [librosa.time_to_samples(t, cow[1]) for t in timestamps_cow]

bounds_cow[0]=bounds_cow[0]-25000
bounds_cow[1]=bounds_cow[0]+35000
print(bounds_cow)
cow_in_F = cow[0][bounds_cow[0]:bounds_cow[1]]
ipd.Audio(cow_in_F, rate=cow[1])

In [ ]:
#for cow
last_int_sample = last_not_0_before_par(bounds_cow[1])
sample = last_int_sample
for _  in range(50):
    sample = sign_change(sample-1)
last_sample_extension = sample

for _  in range(1000):
    sample = sign_change(sample-1)
first_sample_extension = sample
    
#first_sample_extension = sample
print(last_int_sample)
print(first_sample_extension)

long_sample_cow = cow[0][bounds_cow[0]:bounds_cow[1]]
for _ in range(10):
    long_sample_cow = np.concatenate((long_sample_cow, cow[0][first_sample_extension:last_sample_extension]))
print(len(long_sample_cow))
ipd.Audio(long_sample_cow, rate=cow[1])

In [ ]:
#for cow
def long_enough_ignoring_frequencies(num_samples):
    output = cow[0][bounds_cow[0]:bounds_cow[1]]
    while len(output) < num_samples:
        output = np.concatenate((output, cow[0][first_sample_extension:last_sample_extension]))
    return output